# 웹 크롤링 selenium 사용 예

In [1]:
# 라이브러리 불러오기
import pandas as pd
from selenium import webdriver
import time
import math

In [2]:
# 초기 설정

# 블루베리 시나몬 플렉스 시리얼
url = "https://www.coupang.com/vp/products/29528864?vendorItemId=3222696681&sourceType=SDP_ALSO_VIEWED&rmdId=e4655b075bca4d43a084d2fc95cb46d8&eventLabel=recommendation_widget_pc_sdp_001&platform=web&rmdABTestInfo=9266:C,10242:C,8534:A,8088:A,8091:A,9437:A&rmdValue=p1421463742:vt-1.0.0:p29528864&isAddedCart="

# 페이지 넘길 경우 이미지 등 화면 불러오기 기다리는 시간 단위:초
# 컴퓨터나 네트워크 상황에 따라 너무 빨리 데이터를 가져오면 에러가 발생함. 이런 경우 시간을 늘려야 함.
interval = 2 

# 데이터를 담을 리스트
data_list = []


In [3]:
# 크롬 웹드라이버
driver = webdriver.Chrome("./chromedriver")  
driver.get(url)   

WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [4]:
# 상품평 클릭 상품상세에서 상품평으로 이동함. 
driver.find_element_by_css_selector('.count').click()
time.sleep(interval)

NameError: name 'driver' is not defined

In [5]:
# 총 리뷰수 확인
review_total = driver.find_element_by_css_selector('.sdp-review__average__total-star__info-count').text
review_total = review_total.replace(",", "")
review_total

'206'

In [6]:
#페이지별 리뷰 개수
review_per_page = 5

# 전체 페이지 수 계산 리뷰가 9개 일 경우 페이지수는 9 / 5 = 1.8이 나옴
total_page = int(review_total) / review_per_page

# 리뷰가 9개 일 경우 페이지수는 9 / 5 = 1.8 -> 2페이지 올림해야 함. 올림은 ceil
total_page = math.ceil(total_page)

total_page 

42

In [7]:
# 상품명 확인
product = driver.find_element_by_css_selector('.prod-buy-header__title').text
product

'네이쳐패스 블루베리 시나몬 플렉스 시리얼'

In [8]:
# 사용자명과 평점 수집 함수 정의
def get_page_data():

    # 사용자명 수집
    users = driver.find_elements_by_css_selector('.sdp-review__article__list__info__user__name.js_reviewUserProfileImage')
    
    # 평점 수집
    ratings = driver.find_elements_by_css_selector('.sdp-review__article__list__info__product-info__star-orange.js_reviewArticleRatingValue')
        
    # 사용자명수와 평점수가 같을 경우만 수집
    if len(users) == len(ratings):            
        for index in range(len(users)):
            data = {}
            data['username'] = users[index].text
            data['rating'] = int(ratings[index].get_attribute('data-rating'))
            print(data)
            data_list.append(data)        


In [9]:
print("수집 시작")

# 첫 페이지 수집하고 시작
get_page_data() 

# 버튼을 눌러서 페이지를 이동해 가면서 계속 수집.
# 예외처리를 해줘야 함. 하지 않으면 중지됨.
for page in range(1, total_page):
    try:
        print(str(page) + " page 수집 끝")

        button_index = page % 10 + 2

        # 데이터 수집이 끝난 뒤 다음 페이지 버튼을 클릭
        driver.find_element_by_xpath('//*[@id="btfTab"]/ul[2]/li[2]/div/div[5]/section[4]/div[3]/button[' + str(button_index) +']').click()
        time.sleep(interval)

        #1 0page 수집이 끝나서 11로 넘어가기 위해서는 > 버튼을 눌러야 함.
        if(page % 10 == 0):
            driver.find_element_by_xpath('//*[@id="btfTab"]/ul[2]/li[2]/div/div[5]/section[4]/div[3]/button[12]').click()
            time.sleep(interval)

        # 해당 페이지 데이터 수집 
        get_page_data()
    except:
        print("수집 에러")

print(str(page) + " page 수집 끝")
print("수집 종료")

수집 시작
{'username': '웅장해진다 ', 'rating': 5}
{'username': '배*웅 ', 'rating': 2}
{'username': '김*람 ', 'rating': 5}
{'username': '박*름 ', 'rating': 5}
{'username': '팜*늉 ', 'rating': 5}
1 page 수집 끝
{'username': '집콕하쥬 ', 'rating': 5}
{'username': '신*경 ', 'rating': 3}
{'username': '김*정 ', 'rating': 5}
{'username': '장*우 ', 'rating': 4}
{'username': '과자중독자 ', 'rating': 5}
2 page 수집 끝
{'username': '정*란 ', 'rating': 5}
{'username': '박*연 ', 'rating': 5}
{'username': '이*진 ', 'rating': 5}
{'username': '이*연 ', 'rating': 4}
{'username': '사*현 ', 'rating': 5}
3 page 수집 끝
{'username': '김*옥 ', 'rating': 5}
{'username': '김*옥 ', 'rating': 5}
{'username': '이*수 ', 'rating': 5}
{'username': '함*지 ', 'rating': 5}
{'username': '건강한살림 ', 'rating': 5}
4 page 수집 끝
{'username': 'haisue ', 'rating': 5}
{'username': '최*진 ', 'rating': 4}
{'username': '김*옥 ', 'rating': 3}
{'username': '건강한살림 ', 'rating': 5}
{'username': '홍*지 ', 'rating': 5}
5 page 수집 끝
{'username': '김보통 ', 'rating': 5}
{'username': '백*희 ', 'rating': 4}
{'us

In [11]:
df = pd.DataFrame(data_list)
df       

,username,rating
0,웅장해진다,5
1,배*웅,2
2,김*람,5
3,박*름,5
4,팜*늉,5
...,...,...
201,김*애,3
202,배*현,5
203,L*A,5
204,배*현,5


In [12]:
# 엑셀로 저장
df.to_excel("coupang-crawling-example.xlsx")

# 전체 코드

In [16]:
# 라이브러리 불러오기
import pandas as pd
from selenium import webdriver
import time
import math

# 초기 설정

# 블루베리 시나몬 플렉스 시리얼
url = "https://www.coupang.com/vp/products/29528864?vendorItemId=3222696681&sourceType=SDP_ALSO_VIEWED&rmdId=e4655b075bca4d43a084d2fc95cb46d8&eventLabel=recommendation_widget_pc_sdp_001&platform=web&rmdABTestInfo=9266:C,10242:C,8534:A,8088:A,8091:A,9437:A&rmdValue=p1421463742:vt-1.0.0:p29528864&isAddedCart="

# 페이지 넘길 경우 이미지 등 화면 불러오기 기다리는 시간 단위:초
# 컴퓨터나 네트워크 상황에 따라 너무 빨리 데이터를 가져오면 에러가 발생함. 이런 경우 시간을 늘려야 함.
interval = 2 

# 데이터를 담을 리스트
data_list = []

# 크롬 웹드라이버
driver = webdriver.Chrome("./chromedriver")  
driver.get(url)   
time.sleep(interval)

# 상품평 클릭 상품상세에서 상품평으로 이동함. 
driver.find_element_by_css_selector('.count').click()
time.sleep(interval)

# 총 리뷰수 확인
review_total = driver.find_element_by_css_selector('.sdp-review__average__total-star__info-count').text
review_total = review_total.replace(",", "")
print(review_total)

#페이지별 리뷰 개수
review_per_page = 5

# 전체 페이지 수 계산 리뷰가 9개 일 경우 페이지수는 9 / 5 = 1.8이 나옴
total_page = int(review_total) / review_per_page

# 리뷰가 9개 일 경우 페이지수는 9 / 5 = 1.8 -> 2페이지 올림해야 함. 올림은 ceil
total_page = math.ceil(total_page)
print(total_page)

# 상품명 확인
product = driver.find_element_by_css_selector('.prod-buy-header__title').text
print(product)

# 사용자명과 평점 수집 함수 정의
def get_page_data():

    # 사용자명 수집
    users = driver.find_elements_by_css_selector('.sdp-review__article__list__info__user__name.js_reviewUserProfileImage')
    
    # 평점 수집
    ratings = driver.find_elements_by_css_selector('.sdp-review__article__list__info__product-info__star-orange.js_reviewArticleRatingValue')
        
    # 사용자명수와 평점수가 같을 경우만 수집
    if len(users) == len(ratings):            
        for index in range(len(users)):
            data = {}
            data['username'] = users[index].text
            data['rating'] = int(ratings[index].get_attribute('data-rating'))
            print(data)
            data_list.append(data)
            
print("수집 시작")

# 첫 페이지 수집하고 시작
get_page_data() 

# 버튼을 눌러서 페이지를 이동해 가면서 계속 수집.
# 예외처리를 해줘야 함. 하지 않으면 중지됨.
for page in range(1, total_page):
    try:
        print(str(page) + " page 수집 끝")

        button_index = page % 10 + 2

        # 데이터 수집이 끝난 뒤 다음 페이지 버튼을 클릭
        driver.find_element_by_xpath('//*[@id="btfTab"]/ul[2]/li[2]/div/div[5]/section[4]/div[3]/button[' + str(button_index) +']').click()
        time.sleep(interval)

        #1 0page 수집이 끝나서 11로 넘어가기 위해서는 > 버튼을 눌러야 함.
        if(page % 10 == 0):
            driver.find_element_by_xpath('//*[@id="btfTab"]/ul[2]/li[2]/div/div[5]/section[4]/div[3]/button[12]').click()
            time.sleep(interval)

        # 해당 페이지 데이터 수집 
        get_page_data()
    except:
        print("수집 에러")

print(str(page) + " page 수집 끝")
print("수집 종료")
    
df = pd.DataFrame(data_list)
print(df)

# 엑셀로 저장
df.to_excel("coupang-crawling-example.xlsx")



206
42
네이쳐패스 블루베리 시나몬 플렉스 시리얼
수집 시작
{'username': '웅장해진다 ', 'rating': 5}
{'username': '배*웅 ', 'rating': 2}
{'username': '김*람 ', 'rating': 5}
{'username': '박*름 ', 'rating': 5}
{'username': '팜*늉 ', 'rating': 5}
1 page 수집 끝
{'username': '집콕하쥬 ', 'rating': 5}
{'username': '신*경 ', 'rating': 3}
{'username': '김*정 ', 'rating': 5}
{'username': '장*우 ', 'rating': 4}
{'username': '과자중독자 ', 'rating': 5}
2 page 수집 끝
{'username': '정*란 ', 'rating': 5}
{'username': '박*연 ', 'rating': 5}
{'username': '이*진 ', 'rating': 5}
{'username': '이*연 ', 'rating': 4}
{'username': '사*현 ', 'rating': 5}
3 page 수집 끝
{'username': '김*옥 ', 'rating': 5}
{'username': '김*옥 ', 'rating': 5}
{'username': '이*수 ', 'rating': 5}
{'username': '함*지 ', 'rating': 5}
{'username': '건강한살림 ', 'rating': 5}
4 page 수집 끝
{'username': 'haisue ', 'rating': 5}
{'username': '최*진 ', 'rating': 4}
{'username': '김*옥 ', 'rating': 3}
{'username': '건강한살림 ', 'rating': 5}
{'username': '홍*지 ', 'rating': 5}
5 page 수집 끝
{'username': '김보통 ', 'rating': 5}
{'userna